<a href="https://colab.research.google.com/github/pacnpacn/BUGAN/blob/master/script_GANvoxel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#right click shared folder IRCMS_GAN_collaborative_database and "Add shortcut to Drive" to My drive
%cd drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj/generated_files/
!ls

/content/drive/.shortcut-targets-by-id/1ylB2p6N0qQ-G4OsBuwcZ9C0tsqVu9ww4/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj/generated_files
discriminator.pth	  maple_1_voxel_size1.ply   old_2_voxel_size03.ply
generator.pth		  maple_2_voxel_size03.ply  old_3_voxel_size03.ply
maple_1_voxel_size03.ply  maple_3_voxel_size03.ply
maple_1_voxel_size05.ply  old_1_voxel_size03.ply


In [3]:
!pip install open3d

#add open3d to process voxel, mesh, and point cloud

In [4]:
import open3d as o3d
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader, TensorDataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

#dataset

In [5]:
def voxel2arrayCentered(voxel, tree_size_scale = 1):
    array_size = np.array([250, 250, 250])
    vox_array = np.zeros(array_size, dtype=int)  
    tree_size = np.array(voxel.get_axis_aligned_bounding_box().get_extent())
    tree_size = np.ceil(tree_size / tree_size_scale)    #voxel_size = tree_size_scale
    tree_center = (np.ceil(tree_size / 2)).astype(int)

    vox_list = voxel.get_voxels()
    for vox in vox_list:
        coord = vox.grid_index
        #center the tree
        coord = coord - tree_center + (array_size/2)
        coord = tuple(coord.astype(int))

        vox_array[coord] = 1.

    return vox_array

In [6]:
#TODO: process all files in the generated file folder to generate dataset 
# import os

# data = []
# for file_name in os.listdir():
#     if file_name.endswith("voxel_size03.ply"):
#         #note that the voxel_size of vox is 0.3, so we scale it back to one for indexing
#         data.append(voxel2arrayCentered(o3d.io.read_voxel_grid(file_name), 0.3))


In [7]:
dataset = []
dataset.append(voxel2arrayCentered(o3d.io.read_voxel_grid('maple_1_voxel_size03.ply'), 0.3))
# dataset.append(voxel2arrayCentered(o3d.io.read_voxel_grid('maple_2_voxel_size03.ply'), 0.3))
# dataset.append(voxel2arrayCentered(o3d.io.read_voxel_grid('maple_3_voxel_size03.ply'), 0.3))

In [8]:
dataset = torch.tensor(dataset).float()
print(torch.unsqueeze(dataset, -1).shape)
tensor_dataset = TensorDataset(torch.unsqueeze(dataset, 1))
dataloader = DataLoader(tensor_dataset, batch_size=2)

torch.Size([1, 250, 250, 250, 1])


#model description

In [9]:
#input: 128-d noise vector
#output: (250,250,250) array with values in [0,1]

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.gen_fc = nn.Linear(128, 16 * 5 * 5 * 5)
        self.gen = nn.Sequential(
            nn.ConvTranspose3d(16, 32, 5, 2),
            nn.BatchNorm3d(32),
            nn.ReLU(True),
            nn.ConvTranspose3d(32, 32, 5, 2),
            nn.BatchNorm3d(32),
            nn.ReLU(True),
            nn.ConvTranspose3d(32, 32, 5, 2),
            nn.BatchNorm3d(32),
            nn.ReLU(True),
            nn.ConvTranspose3d(32, 32, 3, 2),
            nn.BatchNorm3d(32),
            nn.ReLU(True),
            nn.ConvTranspose3d(32, 16, 3, 2),
            nn.BatchNorm3d(16),
            nn.ReLU(True),
            nn.ConvTranspose3d(16, 1, 4, 1),
            nn.Sigmoid()
        )

    def forward(self, x):

        x = self.gen_fc(x)
        x = x.view(x.shape[0], 16, 5, 5, 5)
        x = self.gen(x)
        return x


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.dis = nn.Sequential(
            nn.Conv3d(1, 16, 5, 1),
            nn.LeakyReLU(0.1, True),
            nn.MaxPool3d((2, 2, 2)),
            nn.Conv3d(16, 32, 5, 1),
            nn.LeakyReLU(0.1, True),
            nn.MaxPool3d((2, 2, 2)),
            nn.Conv3d(32, 32, 5, 1),
            nn.LeakyReLU(0.1, True),
            nn.MaxPool3d((2, 2, 2)),
            nn.Conv3d(32, 16, 5, 1),
            nn.LeakyReLU(0.1, True),
            nn.MaxPool3d((2, 2, 2)),
            nn.Conv3d(16, 16, 5, 1),
            nn.LeakyReLU(0.1, True),
            nn.Conv3d(16, 8, 5, 1),
            nn.LeakyReLU(0.1, True),
        )

        self.dis_fc = nn.Sequential(
            nn.Linear(8 * 3 * 3 * 3, 128),
            nn.LeakyReLU(0.1, True),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):

        x = self.dis(x)
        x = x.view(x.shape[0], -1)
        x = self.dis_fc(x)
        return x

In [10]:
G = Generator().to(device)
summary(G, (128,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 2000]         258,000
   ConvTranspose3d-2       [-1, 32, 13, 13, 13]          64,032
       BatchNorm3d-3       [-1, 32, 13, 13, 13]              64
              ReLU-4       [-1, 32, 13, 13, 13]               0
   ConvTranspose3d-5       [-1, 32, 29, 29, 29]         128,032
       BatchNorm3d-6       [-1, 32, 29, 29, 29]              64
              ReLU-7       [-1, 32, 29, 29, 29]               0
   ConvTranspose3d-8       [-1, 32, 61, 61, 61]         128,032
       BatchNorm3d-9       [-1, 32, 61, 61, 61]              64
             ReLU-10       [-1, 32, 61, 61, 61]               0
  ConvTranspose3d-11    [-1, 32, 123, 123, 123]          27,680
      BatchNorm3d-12    [-1, 32, 123, 123, 123]              64
             ReLU-13    [-1, 32, 123, 123, 123]               0
  ConvTranspose3d-14    [-1, 16, 247, 2

In [11]:
D = Discriminator().to(device)
summary(D, (1, 250, 250, 250))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1    [-1, 16, 246, 246, 246]           2,016
         LeakyReLU-2    [-1, 16, 246, 246, 246]               0
         MaxPool3d-3    [-1, 16, 123, 123, 123]               0
            Conv3d-4    [-1, 32, 119, 119, 119]          64,032
         LeakyReLU-5    [-1, 32, 119, 119, 119]               0
         MaxPool3d-6       [-1, 32, 59, 59, 59]               0
            Conv3d-7       [-1, 32, 55, 55, 55]         128,032
         LeakyReLU-8       [-1, 32, 55, 55, 55]               0
         MaxPool3d-9       [-1, 32, 27, 27, 27]               0
           Conv3d-10       [-1, 16, 23, 23, 23]          64,016
        LeakyReLU-11       [-1, 16, 23, 23, 23]               0
        MaxPool3d-12       [-1, 16, 11, 11, 11]               0
           Conv3d-13          [-1, 16, 7, 7, 7]          32,016
        LeakyReLU-14          [-1, 16, 

#train

In [12]:
def train_model(generator, discriminator, dataloader, lr_gen = 0.0003, lr_dis = 0.0003, num_epochs = 3):   
    
    generator.to(device)
    discriminator.to(device)
    
    #loss function
    criterion = nn.BCELoss(reduction='sum')
    #optimizer
    dis_optimizer = optim.Adam(discriminator.parameters(), lr=lr_dis)
    gen_optimizer = optim.Adam(generator.parameters(), lr=lr_gen)   

    generator.train()
    discriminator.train()

    d_losses = []
    g_losses = []
    for epoch in range(num_epochs):
        d_ep_loss = 0.
        g_ep_loss = 0.
        for dataset_batch in dataloader:
            dataset_batch = dataset_batch[0]
            dataset_batch = dataset_batch.to(device)
            batch_size = dataset_batch.shape[0]
            
            #labels
            real_label = torch.unsqueeze(torch.ones(batch_size),1).to(device)
            fake_label = torch.unsqueeze(torch.zeros(batch_size),1).to(device)

            
############
            #train discriminator
            ############
            #generate fake trees
            z = torch.randn(batch_size, 128).to(device) #128-d noise vector
            tree_fake = generator(z)

            #real data (data from dataloader)
            dout_real = discriminator(dataset_batch)
            dloss_real = criterion(dout_real, real_label)
            score_real = dout_real
            #fake data (data from generator)            
            dout_fake = discriminator(tree_fake)
            dloss_fake = criterion(dout_fake, fake_label)
            score_fake = dout_fake

            #loss function (discriminator classify real data vs generated data)
            dloss = dloss_real + dloss_fake

            #optimize discriminator
            dis_optimizer.zero_grad()
            dloss.backward()
            dis_optimizer.step()

            #record loss
            d_ep_loss += dloss.detach()

            ############
            #train generator
            ############

            #generate fake tree again (not sure)
            z = torch.randn(batch_size, 128).to(device) #128-d noise vector
            tree_fake = generator(z)
            dout_fake = discriminator(tree_fake)
            #generator should generate trees that discriminator think they are real
            gloss = criterion(dout_fake, real_label)    

            #optimize generator
            gen_optimizer.zero_grad() 
            gloss.backward()
            gen_optimizer.step()

            #record loss
            g_ep_loss += gloss.detach()

        #after each epoch, record total loss
        d_losses.append(d_ep_loss)
        g_losses.append(g_ep_loss)
        print("discriminator epoch"+str(epoch)+" : "+str(d_ep_loss))
        print("gen_optimizer epoch"+str(epoch)+" : "+str(g_ep_loss))

        #save model if necessary
        if epoch % 10 == 9:
            torch.save(generator.state_dict(), './generator.pth')
            torch.save(discriminator.state_dict(), './discriminator.pth')



    return generator, discriminator

In [13]:
G = Generator()
D = Discriminator()
G, D = train_model(G, D, dataloader, num_epochs = 30)       #given dataloader has only 1 model, the training time is 72s per epoch.

discriminator epoch0 : tensor(1.3879, device='cuda:0')
gen_optimizer epoch0 : tensor(0.6601, device='cuda:0')
discriminator epoch1 : tensor(1.3859, device='cuda:0')
gen_optimizer epoch1 : tensor(0.6691, device='cuda:0')
discriminator epoch2 : tensor(1.3781, device='cuda:0')
gen_optimizer epoch2 : tensor(0.7013, device='cuda:0')
discriminator epoch3 : tensor(1.3487, device='cuda:0')
gen_optimizer epoch3 : tensor(0.8170, device='cuda:0')
discriminator epoch4 : tensor(1.2631, device='cuda:0')
gen_optimizer epoch4 : tensor(1.1814, device='cuda:0')
discriminator epoch5 : tensor(1.0900, device='cuda:0')
gen_optimizer epoch5 : tensor(2.0367, device='cuda:0')
discriminator epoch6 : tensor(0.8810, device='cuda:0')
gen_optimizer epoch6 : tensor(3.5002, device='cuda:0')
discriminator epoch7 : tensor(0.7126, device='cuda:0')
gen_optimizer epoch7 : tensor(2.5729, device='cuda:0')
discriminator epoch8 : tensor(0.4515, device='cuda:0')
gen_optimizer epoch8 : tensor(6.8215, device='cuda:0')
discrimina